In [1]:
from konlpy.tag import Mecab
import json

In [2]:
mecab = Mecab()

In [3]:
with open('/opt/ml/for_ensemble/nbest_predictions_bert_kor_base.json') as f:
    bert = json.load(f)

In [4]:
with open('/opt/ml/for_ensemble/nbest_predictions_koelectra.json') as f:
    koelec = json.load(f)

In [5]:
result_dict_bert = {}
busa = ['MAG', 'MAJ']
josa=['JKS', 'JKC', 'JKG', 'JKO', 'JKB', 'JKV', 'JKQ', 'JX', 'JC']
buho=['SY']
ami=['EP','EF','EC','ETN','ETM'] #djal

for k in bert:
    for j in range(20):
        tagged = mecab.pos(bert[k][j]['text'])
        if len(tagged) == 1:
            _, tag = tagged[0]
            if tag not in busa:
                result_dict_bert[k] = [bert[k][j]['text'],bert[k][j]['start_logit']+bert[k][j]['end_logit']]
                break
        else:
            temp=''
            word,tag = tagged[-1]
           
            if tag in josa:
                josalen=len(word)
                temp=bert[k][j]['text'][:len(bert[k][j]['text'])-josalen]
                result_dict_bert[k] = [temp,bert[k][j]['start_logit']+bert[k][j]['end_logit']]
            else:
                result_dict_bert[k] = [bert[k][j]['text'],bert[k][j]['start_logit']+bert[k][j]['end_logit']]
            break

In [6]:
result_dict_koelec = {}

for k in koelec:
    for j in range(20):
        jump=koelec[k][j]['text'].split(' ')
        maxjump=len(jump)
        before=-1
        now=0
        
        tagged = mecab.pos(koelec[k][j]['text'])
        if len(tagged) == 1:
            _, tag = tagged[0]
            if tag not in busa:
                result_dict_koelec[k] = [koelec[k][j]['text'],koelec[k][j]['start_logit']+koelec[k][j]['end_logit']]
                break
        else:
            temp=''
            word,tag = tagged[-1]
           
            if tag in josa:
                josalen=len(word)
                temp=koelec[k][j]['text'][:len(koelec[k][j]['text'])-josalen]
                result_dict_koelec[k] = [temp,koelec[k][j]['start_logit']+koelec[k][j]['end_logit']]
            else:
                result_dict_koelec[k] = [koelec[k][j]['text'],koelec[k][j]['start_logit']+koelec[k][j]['end_logit']]
            break

In [7]:
with open('pred_remove_tags_koelec.json', "w") as writer:
    writer.write(json.dumps(result_dict_koelec, indent=4, ensure_ascii=False) + "\n")

In [8]:
with open('pred_remove_tags_bert.json', "w") as writer:
    writer.write(json.dumps(result_dict_bert, indent=4, ensure_ascii=False) + "\n")

In [9]:
final={}

In [10]:
for k in koelec:
    if result_dict_koelec[k][1]>result_dict_bert[k][1]:
        final[k]=result_dict_koelec[k][0]
    else:
        final[k]=result_dict_bert[k][0]

In [11]:
with open('final.json', "w") as writer:
    writer.write(json.dumps(final, indent=4, ensure_ascii=False) + "\n")